In [ ]:
#| default_exp record

In [ ]:
#| export
import queue
import soundfile as sf
import sys
import os
import shutil
import datetime
from audio_journal.sounds import play_start_sound, play_stop_sound

In [ ]:
#| export
def record_audio(filename):
    import sounddevice as sd
    sample_rate = 44100
    channels = 1
    q = queue.Queue()


    def audio_callback(indata, frames, time, status):
        if status:
            print(status, file=sys.stderr)
        q.put(indata.copy())

    try:
        with sf.SoundFile(filename, mode='x', samplerate=sample_rate,
                          channels=channels, subtype='PCM_24') as file:
            with sd.InputStream(samplerate=sample_rate, channels=channels, callback=audio_callback):
                print("Waiting for keyboard input now...")
                input()
                sd.stop()
                while not q.empty():
                    file.write(q.get())
    except Exception as e:
        print(f"Error while recording audio: {e}")

In [ ]:
#| export

def recording_process():
    os.makedirs("temp_recordings", exist_ok=True)
    os.makedirs("completed_recordings", exist_ok=True)
    while True:
        print("Press the space key to start recording...")
        input()
        play_start_sound()
        print("Recording... Press the space key to stop recording.")
        current_date_time = datetime.datetime.now().strftime("%B %d, %Y %H-%M-%S")
        temp_audio_filename = f"temp_recordings/audio_{current_date_time}.wav"
        completed_audio_filename = f"completed_recordings/audio_{current_date_time}.wav"
        record_audio(temp_audio_filename)
        play_stop_sound()
        shutil.move(temp_audio_filename, completed_audio_filename)
        print(f"Audio recorded and saved to {completed_audio_filename}")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()